In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#import comet_ml in the top of your file
from comet_ml import Experiment
import os
import matplotlib.pyplot as plt
import joblib
from copy import deepcopy
import numpy as np
from sklearn.feature_selection import RFECV
import json
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
from sklearn import feature_selection
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import xgboost as xgb
import pandas as pd
from ift6758.features.feature_engineering2 import SeasonDataSetTwo
from ift6758.metrics import plot_metrics
import random
random_state = 42
import tempfile
import sklearn
from ift6758.features.feature_engineering_game import GameData

In [9]:
gd= GameData(gameId=2017021065)

In [10]:
pd.set_option('display.max_columns',None)

In [11]:
gd.get_features_2()

C:\Users\saiak\.conda\envs\ift6758-conda-env\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\saiak\.conda\envs\ift6758-conda-env\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


(    eventId      gamePk            team.name  gamePeriod  coordinatesX  \
 0        11  2017021065  Washington Capitals           1         -50.0   
 1        12  2017021065  Washington Capitals           1         -85.0   
 2        16  2017021065        Winnipeg Jets           1          73.0   
 3        20  2017021065  Washington Capitals           1         -29.0   
 4        21  2017021065        Winnipeg Jets           1          23.0   
 ..      ...         ...                  ...         ...           ...   
 66      355  2017021065  Washington Capitals           4          71.0   
 67      356  2017021065  Washington Capitals           4          58.0   
 68      358  2017021065  Washington Capitals           4          60.0   
 69      364  2017021065  Washington Capitals           4          74.0   
 70      367  2017021065  Washington Capitals           4          74.0   
 
     coordinatesY    shotType emptyNet isHome  gameSeconds  totalGameSeconds  \
 0           36.0 

In [16]:
from ift6758.ift6758.client.game_client import GameClient

In [21]:
gc = GameClient()
gc.ping_game(2017021065,non_processed_event=355)

C:\Users\saiak\.conda\envs\ift6758-conda-env\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\saiak\.conda\envs\ift6758-conda-env\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


(    eventId      gamePk            team.name  gamePeriod  coordinatesX  \
 66      355  2017021065  Washington Capitals           4          71.0   
 67      356  2017021065  Washington Capitals           4          58.0   
 68      358  2017021065  Washington Capitals           4          60.0   
 69      364  2017021065  Washington Capitals           4          74.0   
 70      367  2017021065  Washington Capitals           4          74.0   
 
     coordinatesY    shotType emptyNet isHome  gameSeconds  totalGameSeconds  \
 66          -8.0   Snap Shot        0   True         95.0            3695.0   
 67          10.0   Slap Shot        0   True        106.0            3706.0   
 68          23.0   Slap Shot        0   True        134.0            3734.0   
 69           1.0  Wrist Shot        0   True        207.0            3807.0   
 70           1.0  Wrist Shot    False   True        251.0            3851.0   
 
     shotDistance  shotAngle  isGoal lastEventType  lastEventCoord